### ***Warning***
- This project is currently in experiment
- This classifier does not guarantee the actual status of your image or video
- The definition of "Which is normal, which is abnormal" is not opend in this page
- Please do not use this on your product(In that case, our research group is not responsible)

### ***How to***
- Please install ultralytics in your test machine
- To use GPU, install pytorch for GPU support first and next install ultralytics
- Prepare your video and copy the full path(absolute) or relative path of them
- If the video's running time is too long then you can increase frame_interval so that you can skip test of the frames between interval

In [ ]:
# You are required to install ultralytics first
from ultralytics import YOLO

# Load the attached classifier
model = YOLO("20250922_epoch30.pt")

# Prepare your test videos
# Each video path is an element of the list
video_files = [ ]
# video_files = [ '/path/to/your/video1.mp4',  '/path/to/your/video2.mp4', '/path/to/your/video3.mp4' ]

# Frame sampling interval
frame_interval = 1  # process every 10th frame

## ***Test***
- After prepared, run the code below
- Then you can see the inference output
- The output tuple consists of belows accordingly
  - The path of the video
  - The frame number of the video
  - Inferenced class
  - Actual class
  - Probability

In [ ]:
import cv2
from collections import Counter

predictions = []  # store (video, frame_id, pred_class, true_class)

for video in video_files:
    cap = cv2.VideoCapture(video)
    frame_id = 0

    # ground truth from filename
    true_class = "abnormal" if "abnormal" in video.lower() else "normal"

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_id % frame_interval == 0:
            # run inference
            results = model.predict(frame, imgsz=1088, verbose=False)

            # class names
            names = results[0].names

            # softmax probabilities
            probs = results[0].probs.data.tolist()  # list of floats
            prob_dict = {names[i]: float(p) for i, p in enumerate(probs)}

            # top-1 prediction
            pred_class = names[results[0].probs.top1]

            predictions.append((video, frame_id, pred_class, true_class, prob_dict))
            print((video, frame_id, pred_class, true_class, prob_dict))

        frame_id += 1

    cap.release()

# 3. Frame-level accuracy
correct = sum(1 for (_, _, pred, true, _) in predictions if pred == true)
total = len(predictions)
accuracy = correct / total if total > 0 else 0

print(f"\nFrame-level results")
print(f"Total frames tested: {total}")
print(f"Correct predictions: {correct}")
print(f"Accuracy: {accuracy:.2%}")

# Print sample softmax values
print("\nSample predictions with softmax:")
for v, f, pred, true, prob_dict in predictions[:5]:
    print(f"{v} frame {f}: pred={pred}, true={true}, probs={prob_dict}")

# 4. Video-level (majority vote)
video_results = {}
for video in video_files:
    preds = [pred for (v, _, pred, _, _) in predictions if v == video]
    true_class = "abnormal" if "abnormal" in video.lower() else "normal"
    majority_pred = Counter(preds).most_common(1)[0][0]
    video_results[video] = (majority_pred, true_class)

correct_videos = sum(1 for v, (pred, true) in video_results.items() if pred == true)
accuracy_video = correct_videos / len(video_files)

print("\nVideo-level results:")
for v, (pred, true) in video_results.items():
    print(f"{v}: predicted={pred}, true={true}")
print(f"Video-level accuracy: {accuracy_video:.2%}")

In [ ]:
from collections import defaultdict
import numpy as np

# 3. Frame-level overall accuracy
correct = sum(1 for (_, _, pred, true, _) in predictions if pred == true)
total = len(predictions)
print(f"\nOverall frame-level accuracy: {correct}/{total} = {correct/total:.2%}")

# 4. Video-level detailed stats
print("\nDetailed video-level report:")

video_groups = defaultdict(list)
for entry in predictions:
    video_groups[entry[0]].append(entry)

video_results = {}

for video, entries in video_groups.items():
    total_frames = len(entries)
    correct_frames = sum(1 for (_, _, pred, true, _) in entries if pred == true)
    frame_acc = correct_frames / total_frames if total_frames > 0 else 0

    # Average probabilities across frames
    all_probs = defaultdict(list)
    for (_, _, _, _, prob_dict) in entries:
        for cls, p in prob_dict.items():
            all_probs[cls].append(p)

    avg_probs = {cls: float(np.mean(vals)) for cls, vals in all_probs.items()}

    # Majority vote label
    preds = [pred for (_, _, pred, _, _) in entries]
    from collections import Counter
    majority_pred = Counter(preds).most_common(1)[0][0]
    true_class = entries[0][3]
    correct_video = (majority_pred == true_class)

    video_results[video] = {
        "total_frames": total_frames,
        "correct_frames": correct_frames,
        "frame_accuracy": frame_acc,
        "avg_probs": avg_probs,
        "majority_pred": majority_pred,
        "true_class": true_class,
        "video_correct": correct_video
    }

    # Print per video
    print(f"\n{video}:")
    print(f"  True class       : {true_class}")
    print(f"  Majority predicted: {majority_pred}")
    print(f"  Frames processed : {total_frames}")
    print(f"  Correct frames   : {correct_frames}/{total_frames} ({frame_acc:.2%})")
    print(f"  Average probs    : {avg_probs}")
    print(f"  Video prediction correct? {correct_video}")

In [ ]:
import ast
import pandas as pd
import matplotlib.pyplot as plt

# === 1. Load and parse the log file ===
with open("inference.log", "r", encoding="utf-8") as f:
    raw_lines = f.readlines()

rows = []
for line in raw_lines:
    line = line.strip()
    if not line.startswith("("):  # skip summary lines
        continue
    try:
        filename, frame, pred, true, probs = ast.literal_eval(line)
        rows.append({
            "filename": filename,
            "frame": frame,
            "predicted": pred,
            "true": true,
            "prob_abnormal": probs['abnormal'],
            "prob_normal": probs['normal'],
        })
    except Exception as e:
        print("Skipping line:", line, "Error:", e)

df = pd.DataFrame(rows)

# === 2. Define probability bins ===
bins = [i/10 for i in range(11)]  # 0.0, 0.1, ..., 1.0
labels = [f"{bins[i]:.1f}-{bins[i+1]:.1f}" for i in range(len(bins)-1)]

# === 3. Create bins for abnormal and normal probabilities ===
df['abnormal_bin'] = pd.cut(df['prob_abnormal'], bins=bins, labels=labels, include_lowest=True)
df['normal_bin'] = pd.cut(df['prob_normal'], bins=bins, labels=labels, include_lowest=True)

# === 4. Group by video and probability bin ===
video_dist_abnormal = df.groupby(['filename', 'abnormal_bin']).size().unstack(fill_value=0)
video_dist_normal = df.groupby(['filename', 'normal_bin']).size().unstack(fill_value=0)

# === 5. Print distributions ===
print("\nPer-video probability distribution for ABNORMAL:")
print(video_dist_abnormal)

print("\nPer-video probability distribution for NORMAL:")
print(video_dist_normal)

# === 6. Plot histograms per video ===
for video, subset in df.groupby('filename'):
    # Abnormal
    plt.figure(figsize=(8,5))
    plt.hist(subset['prob_abnormal'], bins=bins, edgecolor='black', alpha=0.7, color='red')
    plt.xlabel("Predicted probability of abnormal")
    plt.ylabel("Frame count")
    plt.title(f"Abnormal probability distribution for {video}")
    plt.xticks(bins)
    plt.tight_layout()
    plt.show()
    
    # Normal
    plt.figure(figsize=(8,5))
    plt.hist(subset['prob_normal'], bins=bins, edgecolor='black', alpha=0.7, color='blue')
    plt.xlabel("Predicted probability of normal")
    plt.ylabel("Frame count")
    plt.title(f"Normal probability distribution for {video}")
    plt.xticks(bins)
    plt.tight_layout()
    plt.show()

# === Plot abnormal probability (red) and normal (blue) ===
for video, subset in df.groupby("filename"):
    plt.figure(figsize=(12, 5))
    
    plt.plot(subset["frame"], subset["prob_abnormal"], label="Abnormal", color="red", alpha=0.7)
    plt.plot(subset["frame"], subset["prob_normal"], label="Normal", color="blue", alpha=0.7)

    plt.xlabel("Frame index")
    plt.ylabel("Probability")
    plt.title(f"Frame-level probabilities for {video}")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()